In [1]:
import os, re, shutil, javalang
import pandas as pd
## https://www.cnblogs.com/lemonbit/p/6810972.html
VERSION_NUMBER_CODE_FORMAT = "R3_2"
VERSION_NUMBER_NATURAL_FORMAT = "3.2"
VERSION_PATH = r"org.eclipse.jdt.core_3.2"
## metrics:
mt = pd.read_csv(r"eclipse-3.2-metrics.csv")
mt["java"] = mt["fileName"].apply(lambda x: x.split(".")[-1] + ".java")
mt["package"] = mt["fileName"].apply(lambda x: ".".join(x.split(".")[:-1]))

In [2]:
mt.rename(columns={
    "numOfClass": "NOT", 
    "numOfInterface": "NOI", 
    "loc": "TLOC", 
    "anonymousClassesQty": "ACD", 
    "numOfStaticMethod": "NSM", 
    "numOfStaticField": "NSF", 
    "numOfMethod": "NOM", 
    "numOfField": "NOF", 
    ### 
    "methodTotalLoc": "MLOC_total", 
    "methodMaxLoc": "MLOC_max", 
    "methodAverageLoc": "MLOC_avg", 
    "totalNestedBlockDepth": "NBD_total", 
    "maxNestedBlockDepth": "NBD_max", 
    "averageNestedBlockDepth": "NBD_avg", 
    "totalParameters": "PAR_total", 
    "maxParameters": "PAR_max", 
    "averageParameters": "PAR_avg", 
    "totalMcCabe": "VG_total", 
    "maxMcCabe": "VG_max", 
    "averageMcCabe": "VG_avg", 
    "totalFanOut": "FOUT_total", 
    "maxFanOut": "FOUT_max", 
    "averageFanOut": "FOUT_avg", 
}, inplace = True)
mt.drop(columns= ["MLOC_avg", "NBD_avg", "PAR_avg", "VG_avg", "FOUT_avg"], inplace= True)
mt.head()

fileName  NOT  NOI  TLOC  ACD  \
0  org.eclipse.jdt.internal.compiler.lookup.Metho...    1    0   516    0   
1                    org.eclipse.jdt.core.IClassFile    0    1    11    0   
2      org.eclipse.jdt.core.search.TypeNameRequestor    1    0     5    0   
3  org.eclipse.jdt.internal.compiler.flow.InsideS...    1    0    35    0   
4  org.eclipse.jdt.internal.core.RenameElementsOp...    1    0    39    0   

   NSM  NSF  NOM  NOF  MLOC_total                    ...                     \
0    0    0   33    7         448                    ...                      
1    0    0    7    0           7                    ...                      
2    0    0    1    0           2                    ...                      
3    0    0    6    1          28                    ...                      
4    0    0    5    0          24                    ...                      

   NBD_total  NBD_max  PAR_total  PAR_max  VG_total  VG_max  FOUT_total  \
0         34        6         49        4       204      44         142   
1         -7       -1          8        3         7       1           0   
2          0        0          5        5         1       1           0   
3          2        2          3        2         9       4           7   
4          0        0          5        4        14       7           8   

   FOUT_max                              java  \
0        20               MethodVerifier.java   
1         0                   IClassFile.java   
2         0            TypeNameRequestor.java   
3         4  InsideSubRoutineFlowContext.java   
4         5      RenameElementsOperation.java   

                                    package  
0  org.eclipse.jdt.internal.compiler.lookup  
1                      org.eclipse.jdt.core  
2               org.eclipse.jdt.core.search  
3    org.eclipse.jdt.internal.compiler.flow  
4             org.eclipse.jdt.internal.core  

[5 rows x 21 columns]

In [3]:
## pre and post bug:

def count_bug(bug_file, version):
    bug_info =  pd.read_csv(bug_file)
    _ = bug_info[["file", "releaseNo", "bugID"]]
    filter_version = _[_["releaseNo"] == version]
    s = filter_version["file"].value_counts()
    bug_num = pd.DataFrame(
        {
            "file": s.index,
            "num_of_bug": s.values
        }
    )
    # bug_num.rename(columns = {"file": "fileName"}, inplace = True)
    return bug_num

post_bug = count_bug(r"../PipeLine/eclipse_post_bug.csv", VERSION_NUMBER_CODE_FORMAT) #
pre_bug = count_bug(r"../PipeLine/eclipse_pre_bug.csv", VERSION_NUMBER_CODE_FORMAT) #
post_bug.head()

file  num_of_bug
0  eclipse\org.eclipse.jdt.core.tests.compiler\sr...          39
1  eclipse\org.eclipse.jdt.core.tests.compiler\sr...          39
2  eclipse\org.eclipse.jdt.core.tests.compiler\sr...          39
3  eclipse\org.eclipse.jdt.core.tests.compiler\sr...          39
4  eclipse\org.eclipse.jdt.core.tests.compiler\sr...          39

### How do we group the file level into package level?
If the package and plugin are the same, the files (.java code files) will be grouped. 

In [4]:
## merge them:
def change_to_package_level(df, group_by_keys):
    """
    Table group, 
    https://www.jb51.net/article/150975.htm
    """
    sum_col = [ i for i in list(df.columns) if "max" not in i]
    sum_col.remove("package")
    sum_col.remove("plugin")
    sum_method = [
        ["sum", "max", "mean"] if i in ["NOT", "NOI", "TLOC", "ACD", "NSM", "NSF", "NOM", "NOF"] else "sum" for i in sum_col
    ]
    sum_col.append("fileName")
    sum_method.append("count")

    max_col = [ i for i in list(df.columns) if "max" in i]
    max_method = ["max" for i in max_col]

    hehe1 = dict(zip(sum_col, sum_method))
    hehe2 = dict(zip(max_col, max_method))
    hehe1.update(hehe2)

    group = df.groupby(group_by_keys)

#     new_table = pd.DataFrame(
#         group.agg(
#             hehe1
#         )
#     ).reset_index()
    
    gp1 = group.agg(
        hehe1
    ).reset_index()
    
    gp2 = gp1.copy(deep=True)
    gp2.columns = gp1.columns.droplevel(0)
    
    gp3 = gp1.copy(deep=True)
    gp3.columns = ["_".join(x) for x in gp3.columns.ravel()]
    gp3.rename(
        columns = {
            "index_": "index",
            "plugin_": "plugin",
            "package_": "package",
            "fileName_count": "NOCU",
            "NOT_sum": "NOT_total",
            "NOT_mean": "NOT_avg",
            "NOI_sum": "NOI_total",
            "NOI_mean": "NOI_avg",
            "TLOC_sum": "TLOC_total",
            "TLOC_mean": "TLOC_avg",
            "ACD_sum": "ACD_total",
            "ACD_mean": "ACD_avg",
            "NSM_sum": "NSM_total",
            "NSM_mean": "NSM_avg",
            "NSF_sum": "NSF_total",
            "NSF_mean": "NSF_avg",
            "NOM_sum": "NOM_total",
            "NOM_mean": "NOM_avg",
            "NOF_sum": "NOF_total",
            "NOF_mean": "NOF_avg",
            "MLOC_total_sum": "MLOC_total",
            "NBD_total_sum": "NBD_total",
            "PAR_total_sum": "PAR_total",
            "VG_total_sum": "VG_total",
            "FOUT_total_sum": "FOUT_total",
            "pre_sum": "pre",
            "post_sum": "post",
            "MLOC_max_max": "MLOC_max",
            "NBD_max_max": "NBD_max",
            "PAR_max_max": "PAR_max",
            "VG_max_max": "VG_max",
            "FOUT_max_max": "FOUT_max",
        }, 
        inplace = True
    )
    
    return gp3

def generate_dataset(prebug, postbug, metricsTable, map_rst_path):
    """
    Merge the table, 
    Column map, 
    """
    file_mapping = pd.read_csv(map_rst_path)
    file_mapping["file"] = file_mapping["file"].map(lambda x: x.replace("#", "\\"))
    _ = pd.merge(metricsTable, file_mapping, on=['java', 'package'], how= "inner")
    _["file"] = _["file"].map(lambda x: x.replace("#", "\\").replace("plugins", "eclipse"))
    # _.to_csv("heheheh.csv", index=False)
    ## temporary file: 
    prebug.rename(columns = {"num_of_bug": "pre"}, inplace = True)
    postbug.rename(columns = {"num_of_bug": "post"}, inplace = True)
    _ = pd.merge(_, prebug, on=['file'], how="left")
    _1 = pd.merge(_, postbug, on=['file'], how="left")
    clean_z = _1["pre"].fillna(0)
    _1["pre"] = clean_z
    clean_z = _1["post"].fillna(0)
    _1["post"] = clean_z
    ## file level
    file_level_table = _1.copy()
    file_level_table.drop(columns=["java", "package", "fileName"], inplace = True)  
    file_level_table["MLOC_avg"] = file_level_table["MLOC_total"] / file_level_table["NOM"]
    file_level_table["NBD_avg"] = file_level_table["NBD_total"] / file_level_table["NOM"]
    file_level_table["PAR_avg"] = file_level_table["PAR_total"] / file_level_table["NOM"]
    file_level_table["VG_avg"] = file_level_table["VG_total"] / file_level_table["NOM"]
    file_level_table["FOUT_avg"] = file_level_table["FOUT_total"] / file_level_table["NOM"]
    # "MLOC_avg", "NBD_avg", "PAR_avg", "VG_avg", "FOUT_avg"
    ## package level
    package = _1.copy()
    package["plugin"] = package["file"].map(lambda x: x.split(os.sep)[1])
    package.drop(columns=["java", "file"], inplace = True)
    package_level = change_to_package_level(package, ["plugin", "package"])
    package_level["MLOC_avg"] = package_level["MLOC_total"] / package_level["NOM_total"]
    package_level["NBD_avg"] = package_level["NBD_total"] / package_level["NOM_total"]
    package_level["PAR_avg"] = package_level["PAR_total"] / package_level["NOM_total"]
    package_level["VG_avg"] = package_level["VG_total"] / package_level["NOM_total"]
    package_level["FOUT_avg"] = package_level["FOUT_total"] / package_level["NOM_total"]
    return file_level_table, package_level

file, package = generate_dataset(pre_bug, post_bug, mt, f"{VERSION_PATH}/org.eclipse.jdt.core_metrics_file_map.csv")

file.to_csv(f"{VERSION_PATH}/dataset-_{VERSION_NUMBER_NATURAL_FORMAT}-filelevel.csv", index=False)
package.to_csv(f"{VERSION_PATH}/dataset-_{VERSION_NUMBER_NATURAL_FORMAT}-packagelevel.csv", index=False)

In [ ]:
file.head() 

In [ ]:
package.head()

### Wasted